# CliMA GCM simple plot example

- this notebook demonstrates simple plots from the GCM output, and the optional use of [Iris 2.4](https://scitools.org.uk/iris/docs/latest/) to manipulate the data. 

In [ ]:
# setup modules 
import numpy as np
from netCDF4 import Dataset
import iris
from scipy import signal
import iris.analysis.cartography
import fnmatch
import os
from matplotlib import animation, rc
from IPython.display import HTML

import iris.plot as iplt
import matplotlib.pyplot as plt
import iris.quickplot as qplt


## Combine files and extract data

In [ ]:
# import data per variable
file_name = 'your_file.nc'

cubes_all = iris.load(file_name)

print(cubes_all)

In [ ]:
# coord info
lo = cubes_all[5].coord('longitude')[:]
la = cubes_all[5].coord('latitude')[:]
ra = cubes_all[5].coord('level')[:]
time = cubes_all[5].coord('time')[:]

lat = np.array(la.points)
lon = np.array(lo.points)
rad = np.array(ra.points)


u_cube = iris.load_cube(file_name,'eastward_wind')
air_cube = iris.load_cube(file_name,'air_temperature')
pot_air_cube = iris.load_cube(file_name,'air_potential_temperature')

print(u_cube)


## Zonal mean view

### 1. Time mean plots

In [ ]:
# temperature and zonal wind

plt.figure(figsize=(15,4))
plt.subplot(141)
zonal_wind = u_cube.collapsed(['time','longitude'], iris.analysis.MEAN)

qplt.contourf( zonal_wind ,20, coords=["latitude","level"])

plt.subplot(142)
air_temp = (air_cube[1]).collapsed(['time','longitude'], iris.analysis.MEAN)
qplt.contourf(air_temp,20, coords=["latitude","level"])

plt.subplot(143)
pot_temp = (pot_air_cube[4]).collapsed(['time','longitude'], iris.analysis.MEAN)
qplt.contourf(pot_temp,20, coords=["latitude","level"])

plt.tight_layout()


### 2. Animation

In [ ]:
var_anim = u_cube.collapsed(['longitude'], iris.analysis.MEAN)
print np.shape((var_anim.data))


In [ ]:
%%capture
%matplotlib inline

# Choose variable
var_anim = u_cube.collapsed(['longitude'], iris.analysis.MEAN)

print np.shape((var_anim.data))

# Initialise plot
rnge = np.arange(-20,21,1)                                   
x,y = np.meshgrid(lat,rad)
fig = plt.figure() 
plt.xlabel(r'lat')
plt.ylabel(r'rad')
init = plt.contourf(lat,rad, var_anim.data[0,:,:]*np.nan, rnge); plt.colorbar(init)

t = np.array(time.points)/60./60./24.
             
# animation function
def update(i): 
    var_one = var_anim.data[i,:,:]
    cont = plt.contourf(lat,rad, var_one, rnge)
    plt.title(r'day %s' %t[i])    
    return cont  

# create animatoin
anim = animation.FuncAnimation(fig, update, frames = np.arange(len(t)), interval=200)

# enable inline display in the notebook
HTML(anim.to_jshtml())
rc('animation', html='jshtml')

In [ ]:
# display
anim

## Horizontal plane view

In [ ]:
# e.g. get upper level, say 10 km altitude
cube_upper = u_cube.extract(iris.Constraint(level=10000))

print cube_upper

### 1. Animation

In [ ]:
%%capture
%matplotlib inline

# Choose variable and its range
var_anim = cube_upper.data 

# pick the appropriate range for contourf
rnge = np.linspace(-1,3,20) # u
#rnge = np.linspace(310,340,20)# temp
#rnge = np.linspace(-9e-6,9e-6,20)# vort

# Initialise plot
x,y = np.meshgrid(lon,lat)
fig = plt.figure() 
plt.xlabel(r'lon')
plt.ylabel(r'lat')
init = plt.contourf(x, y, var_anim[0,:,:]*np.nan, rnge); plt.colorbar(init)

t = np.array(time.points)/60./60./24.

# animation function
def update(i): 
    z = var_anim[i,:,:]
    cont = plt.contourf(x, y, z, rnge)
    plt.title(r'day %s' % t[i])    
    return cont  

# create animatoin
anim = animation.FuncAnimation(fig, update, frames = np.arange(len(t)), interval=200)

# enable inline display in the notebook
HTML(anim.to_jshtml())
rc('animation', html='jshtml')

In [ ]:
# display
anim

### 2. Vorticity test
- the voticity model output is calculated in DG space, allowing for more accurate gradient calculation. As a sanity check, here we compare it to vorticity calculated more crudely from velocities projected on the interpolated grid.
- NB: ```iris.analysis.calculus``` offers a better functionality (e.g. curl func) but needs more tinkering

In [ ]:
# reload cleanly the cube with all vars and get vort, u and v
v_cube = iris.load_cube(file_name,'northward_wind')
vort_cube = iris.load_cube(file_name,'vertical component of relative velocity')

u_cube_up=u_cube.extract(iris.Constraint(level=10000))
v_cube_up=v_cube.extract(iris.Constraint(level=10000))
vort_cube_up=vort_cube.extract(iris.Constraint(level=10000))

# select timestep to plot
sel_time = -1

lfn = len(file_names)
u = (u_cube_up.data)[sel_time,:,:]
v = (v_cube_up.data)[sel_time,:,:]
vortrel = (vort_cube_up.data)[sel_time,:,:]

# range for contourf
rng = np.linspace(-1e-7,1e-7,20)


In [ ]:
# plot 
plt.figure(figsize=(14,4))

# DG vorticity
plt.subplot(121)
c = plt.contourf(lon,lat,(vortrel),rng)
plt.ylabel('lat / deg')
plt.xlabel('lon / deg')
plt.colorbar(c)
plt.title('rel vorticity (DG)')

# compute and plot vorticity on spherical interpolated grid
plt.subplot(122)
coslat = np.cos(lat*np.pi/180.)
dlat = np.abs(lat[2]-lat[1])
dlon = np.abs(lon[2]-lon[1])
vortrel_test = np.gradient( v/(6371000.*coslat[:,None]) ,dlon*np.pi/180.,axis=1) - np.gradient( u * coslat[:,None],dlat*np.pi/180.*6371000., axis=0)/coslat[:,None]
c = plt.contourf(lon,lat,(vortrel_test),rng)
plt.xlabel('lon / deg')
plt.colorbar(c)
plt.title('rel vorticity (interp grid)')
plt.tight_layout()

# add a plot of u and v for reference
plt.figure(figsize=(14,4))
plt.subplot(121)
c = plt.contourf(lon,lat,(np.array(u)[:,:]),20)
plt.ylabel('lat / deg')
plt.xlabel('lon / deg')
plt.colorbar(c)
plt.title('u')
plt.subplot(122)
c = plt.contourf(lon,lat,(np.array(v)[:,:]),20)
plt.ylabel('lat / deg')
plt.xlabel('lon / deg')
plt.colorbar(c)
plt.title('v')